In [ ]:
!pip install Flask faiss-cpu numpy requests openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import faiss
import requests
import openai
from flask import Flask, request, jsonify
from google.colab import drive
from threading import Thread
import os

In [ ]:
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = "....................."

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
# Initialize Flask app
app = Flask(__name__)

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the saved chunks and FAISS index
chunks_path = '/content/drive/MyDrive/ChatBot/Output_Embeddings/chunks.json'
embeddings_path = '/content/drive/MyDrive/ChatBot/Output_Embeddings/embeddings.npy'
index_path = '/content/drive/MyDrive/ChatBot/faiss_index.bin'

In [ ]:
# Load chunks from JSON file
with open(chunks_path, 'r') as f:
    chunks = json.load(f)

In [ ]:
# Load embeddings and FAISS index
embeddings = np.load(embeddings_path)
index = faiss.read_index(index_path)

In [ ]:
def get_query_embedding(query):
    url = "https://api.openai.com/v1/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
    }
    data = {
        "input": query,
        "model": "text-embedding-3-large"  # Use the appropriate model name for embeddings
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['data'][0]['embedding']
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

In [ ]:
def retrieve_relevant_texts(query_embedding, k=5):
    D, I = index.search(np.array([query_embedding]).astype('float32'), k)
    relevant_texts = [chunks[i] for i in I[0]]  # Get relevant chunks based on indices
    return relevant_texts



In [ ]:
# def generate_diagnosis(query, relevant_texts):
#     prompt = (
#         f"User query: {query}\n\n"
#         f"Based on the following relevant texts: {relevant_texts}, "
#         "provide a likely diagnosis, DSM-IV code, ICD-10 code, and interventions.\n\n"
#         "Here is the likely diagnosis based on the information you have provided:\n"
#         "Diagnosis: \n"
#         "DSM-IV Code: \n"
#         "ICD-10 Code: \n"
#         "Interventions:\n"
#         "- "
#     )
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": prompt}
#         ],
#         max_tokens=150
#     )
#     return response.choices[0].message.content.strip()


In [ ]:
def generate_diagnosis(query, relevant_texts):
    prompt = (
        f"User query: {query}\n\n"
        f"Based on the following relevant texts: {relevant_texts}, "
        "provide a likely diagnosis, DSM-IV code, ICD-10 code, and interventions.\n\n"
        "Here is the likely diagnosis based on the information you have provided:\n"
        "Diagnosis: \n"
        "DSM-IV Code: \n"
        "ICD-10 Code: \n"
        "Interventions:\n"
        "- "
    )

    response_text = ""
    while True:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            stop=None,
        )
        response_content = response.choices[0].message.content.strip()
        response_text += response_content
        if len(response_content) < 150:
            break
        prompt += response_content  # Add previous completion to the prompt to continue

    return response_text


In [ ]:
@app.route('/chatbot', methods=['GET'])
def chatbot():
    query = request.args.get('query')

    # Generate embedding for the user query
    query_embedding = get_query_embedding(query)
    if query_embedding is None:
        return jsonify({'error': 'Failed to generate embedding for the query.'}), 500

    # Retrieve relevant texts using FAISS
    relevant_texts = retrieve_relevant_texts(query_embedding)

    # Generate response based on the retrieved texts
    response_text = generate_diagnosis(query, relevant_texts)

    return jsonify({
        'query': query,
        'response': response_text
    })


In [ ]:
import time

In [ ]:

def run_app():
    app.run(port=5000, debug=False, use_reloader=False)

# Run the Flask app in a separate thread
Thread(target=run_app).start()

# Give the server a moment to ensure it's up and running
time.sleep(5)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import requests

# Example query to test the chatbot endpoint
# test_query = "Client experienced a traumatic event in which he was robbed at gunpoint months ago. Client consistently thinks about this event, experiencing flashbacks and nightmares and is finding it hard to leave his house due to fear that it will happen again. Client is avoiding returning to work, getting groceries or seeing family members. Client stated he is scared of everything and has disturbing thoughts, racing heart rate and stated he jumps at any small noise in his house and is having trouble sleeping. Client stated sometimes he feels he has trouble catching his breath."
test_query = "Client stated that they are constantly stressed and cannot sleep. Always worrying about work and having trouble sleeping."


# Define the URL of the running Flask app
url = "http://127.0.0.1:5000/chatbot"

# Wait until the server is ready
for i in range(5):
    try:
        response = requests.get(url, params={"query": test_query})
        print(response.json())
        break
    except requests.ConnectionError:
        print(f"Attempt {i+1}: Server not yet ready, retrying...")
        time.sleep(2)


INFO:werkzeug:127.0.0.1 - - [31/Jul/2024 18:31:16] "GET /chatbot?query=Client+stated+that+they+are+constantly+stressed+and+cannot+sleep.+Always+worrying+about+work+and+having+trouble+sleeping. HTTP/1.1" 200 -


{'query': 'Client stated that they are constantly stressed and cannot sleep. Always worrying about work and having trouble sleeping.', 'response': 'Diagnosis: Generalized Anxiety Disorder\nDSM-IV Code: 300.02\nICD-10 Code: F41.1\nInterventions: \n1. Cognitive Behavioral Therapy (CBT) to address negative thought patterns and worries.\n2. Relaxation techniques such as deep breathing exercises or progressive muscle relaxation.\n3. Encourage regular exercise and physical activity to reduce stress levels.\n4. Explore mindfulness or meditation practices to promote relaxation and better sleep.\n5. Consider referral to a psychiatrist for evaluation of potential medication for anxiety symptoms.Based on the information you provided, it seems like the client may be experiencing symptoms of Generalized Anxiety Disorder (GAD).\n\nHere are the details:\n- Diagnosis: Generalized Anxiety Disorder\n- DSM-IV Code: 300.02\n- ICD-10 Code: F41.1\n\nInterventions that may be helpful for managing the client\